In [1]:
import osmnx as ox
import networkx as nx
from datamanipulation.divide_net import divide_net
from linkprediction.resource_allocation import RA
from linkprediction.local_random_walk import LRW
from linkprediction.stochastic_block_model import SBM
import numpy as np
from tqdm import tqdm_notebook

In [2]:
wroclaw = ox.io.load_graphml('./data/data_train/Wroclaw_Polska_recent.xml')
nx.readwrite.write_edgelist(wroclaw, './data/data_train/Wroclaw_Polska_recent.csv')
sparse_matrix = nx.convert_matrix.to_scipy_sparse_array(wroclaw)

In [4]:
train, test = divide_net(sparse_matrix, .9)

#nodedegree = np.sum(train, axis=1)
#tempauc = RA(train, test, nodedegree)
#print(f"RA auc is equal to: {tempauc}")
#tempauc = LRW(train, test, 3, 0.85)
#print(f"LRW with 3 steps auc is equal to: {tempauc}")


In [ ]:
np.matrix(train).astype(np.bool_)

In [9]:
tempauc = SBM(np.matrix(train).astype(np.double), np.matrix(test).astype(np.double), 12.0)

In [33]:
from urban import ipdc
import traceback

import csv
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

cities = {}

with open('./cities_w_cords.csv') as f:
        csvreader = csv.DictReader(f, delimiter=';')
        for row in csvreader:
            cities[row['placeid']] = {}
            for field in csvreader.fieldnames[1:]:
                cities[row['placeid']][field] = row[field]   

In [34]:
cities['zurich']

{'nominatimstring': 'zurich',
 'countryid': 'che',
 'name': 'Zurich',
 'lat': '47.3786',
 'lng': '8.54'}

In [10]:
# SETUP

%run -i software/FixBike/packages.py

# import parameters for plotting
%run -i software/FixBike/parameters_plot.py

In [15]:
# IMPORT OBJECTS FROM PREVIOUS STEPS

H = nx.read_gpickle("./data/zurich/pickle/H.gpickle")
gaps_final = pd.read_pickle("./data/zurich/pickle/mygaps.pickle")

# NX attribute dicts (for plotting)
ced = nx.get_edge_attributes(H, "coord") # coordinates of edges dictionary ced
ted = nx.get_edge_attributes(H, "category_edge") # type of edges dictionary ted
tnd = nx.get_node_attributes(H, "category_node") # type of nodes dictionary tnd
cnd = nx.get_node_attributes(H, "coord") # coordinates of nodes dictionary cnd

In [16]:
gaps_final.columns

Index(['path', 'length', 'path_nx', 'o_nx', 'd_nx', 'gapcoord', 'length_b',
       'detour', 'B_star', 'B'],
      dtype='object')

In [ ]:
### SET UP MAP WITH FOLIUM

# make map object

# coopenhagen coordinates
mycity_coord = [55.6761, 12.5683]

m = folium.Map(location = mycity_coord, zoom_start = 13, tiles = None) 

# add tile layers to m
for key in ["osm", "st_lite_op3", "white_background", "Google Satellite"]:
    basemaps[key].add_to(m)
    
##### plot car, onlybike, and multinetworks #####
snw = folium.FeatureGroup(name = "All streets", show = True)
bnw = folium.FeatureGroup(name = "Bikeable", show = True) # bikeable edges 

sloc = []
bloc = []

for edge in H.edges:
    myloc = [(c[1], c[0]) for c in ced[tuple(sorted(edge))].coords]
    sloc.append(myloc)
    if ted[edge] != "car":
        bloc.append(myloc)
        
snw_line = folium.PolyLine(locations = sloc, weight = 2, color = "#dadada").add_to(snw)
bnw_line = folium.PolyLine(locations = bloc, weight = 3, color = "black").add_to(bnw)

snw.add_to(m)
bnw.add_to(m)


#### ADD TO MAP 


gaps_fg = folium.FeatureGroup("Declustered gaps", show = True)

for locs in gap_dec["coord"]:
    myline = folium.PolyLine(locations = locs,
                                weight = 6,
                                color = "black").add_to(gaps_fg) # black border
    # + randomly colored gaps (not classified yet):
    myline = folium.PolyLine(locations = locs,
                                weight = 4,
                                color = random.choice(["#33FFDA", "#0AB023", "#B00A60", "#0A3CB0"])).add_to(gaps_fg)

gaps_fg.add_to(m)


my_fg_nr = folium.FeatureGroup("Gap numbers", show = False)

# Add pop-ups with gap number

for i in range(len(gap_dec)):
    
    my_marker = folium.Marker(location = gap_dec.loc[i]["coord"][0][0], 
                                    popup = "Gap " + str(i)
                             ).add_to(my_fg_nr)
                  
my_fg_nr.add_to(m)

### ADD LAYER CONTROL AND SAVE / DISPLAY FOLIUM MAP
folium.LayerControl().add_to(m)  

m.save("./analysis/gaps_declustered_plot.html")